<a href="https://colab.research.google.com/github/RafsanJany-44/ARC_5SSC_Experiment/blob/main/SleepStage_EEG_EXperiment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Function

In [ ]:
!pip install imbalanced-learn
!pip install ipywidgets

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
import ipywidgets as widgets
from joblib import Parallel, delayed
from google.colab import files
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


def dataset_balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)


def Xy_balance(X_temp, y_temp):
  smote = SMOTE()
  return smote.fit_resample(X_temp, y_temp)


def save_model(model,model_name):
 joblib.dump(model, model_name)
  

def load_model(model_name):
  return joblib.load(model_name)


def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  return dic



def divide_dataset(dataset,percentage,random_state = 0):
  rows,cols = dataset.shape
  if random_state == 0:
    return dataset.iloc[:int(rows*(percentage)),:]
  else:
    import random
    l_r = random.sample(range(0, rows), int(rows*(percentage)))
    return dataset.iloc[l_r,:]



def StandardScaleData(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return  scaler.transform(data)



def MinMaxScaleData(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)


def tune_report_csv(model,dic,feat_selection_method,number_of_feat):

  tunning_type = []
  accuracy = []
  scaled_accuracy = []
  model_name = []
  feature_selection_method = []
  number_of_features = []

  li = ["Smote : NO    ; Scaling : Train(Standard)",
      "Smote : NO    ; Scaling : Train(MinMax)",
      "Smote : Train ; Scaling : Train(Standard)",
      "Smote : Train ; Scaling : Train(MinMax)",
      "Smote : All   ; Scaling : Train(Standard)",
      "Smote : All   ; Scaling : Train(MinMax)"]

  for i in list(dic.keys()):
    X_train, X_test, y_train, y_test = dic[i]
    
    if i in li and str(model[:3]) == "xgb":
      model.fit(X_train,y_train.values)
      y_pred = model.predict(X_test.values)
    else:
      model.fit(X_train,y_train)
      y_pred = model.predict(X_test)

    accu =accuracy_score(y_test, y_pred)
    scaled_accu =  float("{:.2f}".format(accu))
    print(i)
    print("Accurecy: ",scaled_accu)
    tunning_type.append(i)
    accuracy.append(accu)
    scaled_accuracy.append(scaled_accu)
    model_name.append(model)
    feature_selection_method.append(feat_selection_method)
    number_of_features.append(number_of_feat)
    
    print("------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------")

  csv_dic={
      "model_name"  : model_name,
      "tunning_type" : tunning_type,
      "accurecy"    : accuracy, 
      "scaled_accuracy" :scaled_accuracy,
      "feature_selection_method" : feature_selection_method,
      "number_of_features" : number_of_features
  }


  df = pd.DataFrame.from_dict(csv_dic)
  df.to_csv("/content/drive/MyDrive/Sleep Stage 5 class/Features_selection_result/"+feat_selection_method+"_"+str(number_of_feat)+"_"+str(model)+".csv",index = False)
  return "Tunning Data Report is Secured for this phase!!!"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Starting

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/Sleep Stage 5 class/EEG_HMC_FeatureExtraction_2023.01.19.csv")
print(dataset.shape)
dataset.dropna(axis = 0,how='any', inplace = True)
dataset = dataset.reset_index(drop = True)
"""df.to_csv("New_EEG_NullValueRemoved_HMC.csv",index = False)
dataset = pd.read_csv("New_EEG_NullValueRemoved_HMC.csv")"""
target = "Sleep Stage"
result = {}
print(dataset.shape)

#Encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

#dataset = divide_dataset(dataset,0.04611,1)
#print(dataset.shape)

(108920, 52)
(108450, 52)


###Spliting into X and y

In [ ]:
X =  dataset.loc[:,dataset.columns != target]  # removing Sleep Stage
X =  X.loc[:,X.columns != "Subject"]            # removing Status
X =  X.loc[:,X.columns != "Epoch"]             # removing Epoch
y = dataset[target]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108450 entries, 0 to 108449
Data columns (total 49 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   MeanP_Alpha_F4           108450 non-null  float64
 1   MeanP_Beta_F4            108450 non-null  float64
 2   MeanP_Theta_F4           108450 non-null  float64
 3   MeanP_Delta_F4           108450 non-null  float64
 4   MeanP_Gamma_F4           108450 non-null  float64
 5   MeanP_Alpha_C4           108450 non-null  float64
 6   MeanP_Beta_C4            108450 non-null  float64
 7   MeanP_Theta_C4           108450 non-null  float64
 8   MeanP_Delta_C4           108450 non-null  float64
 9   MeanP_Gamma_C4           108450 non-null  float64
 10  MeanP_Alpha_O2           108450 non-null  float64
 11  MeanP_Beta_O2            108450 non-null  float64
 12  MeanP_Theta_O2           108450 non-null  float64
 13  MeanP_Delta_O2           108450 non-null  float64
 14  Mean

#Feature Selection

In [68]:
number_of_feat = 25

###ANOVA with f classifciation

In [ ]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pandas as pd



fs = SelectKBest(score_func=f_classif, k=5)
fit = fs.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_ANOVA'] 

lyst = featureScores.nlargest(number_of_feat,'Score_ANOVA')

#lyst.to_csv('Filter_Method_ANOVA_with_f_classif.csv')

list_of_feat = list(lyst["Best_columns"])
selection_method = "ANOVA"

###Embedded Method

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

imp_coef = coef.sort_values()

list_of_feat=[]


for i in range(coef.shape[0]):
  if coef[i]!=0:
    list_of_feat.append(dataset.iloc[:0,i+3].name)
    
df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Embedded_Method.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]
selection_method = "Embedded"

Best alpha using built-in LassoCV: 1664496.378245
Best score using built-in LassoCV: 0.000418
Lasso picked 4 variables and eliminated the other 45 variables


###Pearson's with f regression

In [69]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import pandas as pd


fs = SelectKBest(score_func=f_regression, k=5)
fit = fs.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_pearsons'] 


lyst = featureScores.nlargest(number_of_feat,'Score_pearsons')

#lyst.to_csv('Filter_Method_Pearson’s_with_f_regression.csv')

list_of_feat = list(lyst["Best_columns"])
selection_method = "Pearson"

###Sequential Feature Selection

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(knn, n_features_to_select=number_of_feat)
sfs.fit(X, y)
list_of_feat=[]
list_of_feat=list(sfs.get_feature_names_out(X.columns))

df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Filter_Method_Sequential_feat_Selection_KNN.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]

selection_method = "Sequential"

#Feature list

In [70]:
X=dataset[list_of_feat]
y=dataset[target]
print(X.shape)

(108450, 25)


In [71]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108450 entries, 0 to 108449
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Relative_MeanP_Beta_C4   108450 non-null  float64
 1   Relative_MeanP_Beta_G    108450 non-null  float64
 2   Relative_MeanP_Beta_O2   108450 non-null  float64
 3   Relative_MeanP_Beta_F4   108450 non-null  float64
 4   Relative_MeanP_Delta_C4  108450 non-null  float64
 5   Relative_MeanP_Alpha_O2  108450 non-null  float64
 6   Relative_MeanP_Delta_G   108450 non-null  float64
 7   Relative_MeanP_Alpha_C4  108450 non-null  float64
 8   Relative_MeanP_Alpha_G   108450 non-null  float64
 9   Relative_MeanP_Delta_O2  108450 non-null  float64
 10  Relative_MeanP_Delta_F4  108450 non-null  float64
 11  Relative_MeanP_Alpha_F4  108450 non-null  float64
 12  Relative_MeanP_Gamma_F4  108450 non-null  float64
 13  MeanP_Delta_O2           108450 non-null  float64
 14  Mean

#Tunning for Scalling and Data Balancing

##Smote : NO
##Scaling : NO

In [72]:
def tune_1():
  X_new = X
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (Standard)

In [73]:
def tune_2():
  X_new = StandardScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (MinMax)

In [74]:
def tune_3():
  X_new = MinMaxScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : Train (Stadard)

In [75]:
def tune_4():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : NO
##Scaling : Train (MinMax)

In [76]:

def tune_5():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : ALL
##Scaling : NO

In [77]:

def tune_6():
  X_new,y_new = Xy_balance(X,y)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)


##Smote : Train
##Scaling : NO

In [78]:

def tune_7():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  return X_train, X_test, y_train, y_test


##Smote : Train
##Scaling : Train (Standard)

In [79]:
def tune_8():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : Train
##Scaling : Train (MixMax)

In [80]:
def tune_9():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : All
##Scaling : Train (Standard)

In [81]:
def tune_10():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : Train (MixMax)

In [82]:
def tune_11():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : All (Standard)

In [83]:
def tune_12():
  X_new,y_new = Xy_balance(X,y)
  X_new = StandardScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : All
##Scaling : ALL (MinMax)

In [84]:
def tune_13():
  X_new,y_new = Xy_balance(X,y)
  X_new = MinMaxScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Tunn Dictionary

In [85]:
tune_dic={
  "Smote : NO    ; Scaling : NO"                  : tune_1(),
  "Smote : NO    ; Scaling : All Data(Standard)"  : tune_2(),
  "Smote : NO    ; Scaling : All Data(MinMax)"    : tune_3(),
  #"Smote : NO    ; Scaling : Train(Standard)"     : tune_4(),
  #"Smote : NO    ; Scaling : Train(MinMax)"       : tune_5(),
  "Smote : ALL   ; Scaling : NO"                  : tune_6(),
  "Smote : Train ; Scaling : NO"                  : tune_7(),
  #"Smote : Train ; Scaling : Train(Standard)"     : tune_8(),
  #"Smote : Train ; Scaling : Train(MinMax)"       : tune_9(),
  #"Smote : All   ; Scaling : Train(Standard)"     : tune_10(),
  #"Smote : All   ; Scaling : Train(MinMax)"       : tune_11(),
  "Smote : All   ; Scaling : All(Standard)"       : tune_12(),
  "Smote : All   ; Scaling : All(MinMax)"        : tune_13()
}



---



# **Training**

##ADABOOST

In [ ]:
%%time
from sklearn.ensemble import AdaBoostClassifier
ada_defult = AdaBoostClassifier(random_state=0)
tune_report_csv(ada_defult,tune_dic,selection_method,number_of_feat)


Smote : NO    ; Scaling : NO
Accurecy:  0.57
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.57
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.57
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.54
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.51
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'



---



---



##Graddient Boosting

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
gradBoost_default = GradientBoostingClassifier(random_state=0)

tune_report_csv(gradBoost_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.65
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.65
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.65
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.63
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.6
------------------------------------------------------------------------------
-----------------------------------

'Tunning Data Report is Secured for this phase!!!'

###Histogram-Based Gradient Boosting

In [ ]:
%%time
from sklearn.ensemble import HistGradientBoostingClassifier

hisgradBoost_default = HistGradientBoostingClassifier(random_state=0)

tune_report_csv(hisgradBoost_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.7
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.7
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.7
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.73
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.68
------------------------------------------------------------------------------
-------------------------------------

'Tunning Data Report is Secured for this phase!!!'



---



---



##Random Forest

In [ ]:
%%time
RanFor={}

from sklearn.ensemble import RandomForestClassifier
rf_default = RandomForestClassifier(random_state=0,n_jobs=-1)

tune_report_csv(rf_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.72
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.72
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.72
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.81
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.7
------------------------------------------------------------------------------
-----------------------------------

'Tunning Data Report is Secured for this phase!!!'



---



---



##XGB

In [ ]:
%%time
import xgboost as xgb
xgb_deafult = xgb.XGBClassifier(random_state=0,n_jobs=-1)

tune_report_csv(xgb_deafult,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.63
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.63
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.63
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.61
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.59
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'

##KNN

In [64]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn_default = KNeighborsClassifier(n_jobs = -1)

tune_report_csv(knn_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.55
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.63
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.61
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.71
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.49
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'

##NB

In [65]:
%%time
from sklearn.naive_bayes import GaussianNB

nb_deafult = GaussianNB()

tune_report_csv(nb_deafult,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.32
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.32
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.32
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.36
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.32
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'

##Tree algorithms

###DecisionTreeRegressor

In [66]:
%%time
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
tune_report_csv(dtc,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.59
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.59
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.59
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.68
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.58
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'

In [67]:
!pip install catboost

from catboost import CatBoostClassifier
#cat = CatBoostClassifier(task_type="GPU")
cat=CatBoostClassifier()

tune_report_csv(cat,tune_dic,selection_method,number_of_feat)

Streaming output truncated to the last 5000 lines.
24:	learn: 1.0386237	total: 3.17s	remaining: 2m 3s
25:	learn: 1.0334126	total: 3.3s	remaining: 2m 3s
26:	learn: 1.0285806	total: 3.4s	remaining: 2m 2s
27:	learn: 1.0232388	total: 3.53s	remaining: 2m 2s
28:	learn: 1.0186090	total: 3.66s	remaining: 2m 2s
29:	learn: 1.0146563	total: 3.79s	remaining: 2m 2s
30:	learn: 1.0119073	total: 3.89s	remaining: 2m 1s
31:	learn: 1.0079719	total: 4s	remaining: 2m 1s
32:	learn: 1.0045896	total: 4.17s	remaining: 2m 2s
33:	learn: 1.0013338	total: 4.34s	remaining: 2m 3s
34:	learn: 0.9982935	total: 4.55s	remaining: 2m 5s
35:	learn: 0.9946931	total: 4.78s	remaining: 2m 8s
36:	learn: 0.9914297	total: 5.01s	remaining: 2m 10s
37:	learn: 0.9880182	total: 5.26s	remaining: 2m 13s
38:	learn: 0.9854396	total: 5.47s	remaining: 2m 14s
39:	learn: 0.9831103	total: 5.66s	remaining: 2m 15s
40:	learn: 0.9801108	total: 5.91s	remaining: 2m 18s
41:	learn: 0.9776449	total: 6.12s	remaining: 2m 19s
42:	learn: 0.9761237	total: 6.

'Tunning Data Report is Secured for this phase!!!'